In [7]:
import numpy as np
from scipy.sparse import diags, hstack, vstack, block_diag
import math

def dft_list(N, pow5):
    length = N << 3
    k = np.array([i for i in range(N)])
    return np.exp(2j * np.pi * pow5[k] / length)

def pow5_list(N):
    length = N << 3
    pow5 = np.arange(N)
    pow5[0] = 1
    for i in range(1, N):
        pow5[i] = 5 * pow5[i - 1]
        pow5[i] = pow5[i] % (length)
        # print(pow5[i])

    return pow5

N = 1 << 12
slot = N // 2

log2_value = math.log2(slot) 
depth = int(log2_value)

V_list = [None] * depth

for i in range(depth):
    block_num = 1 << i
    diag_length = slot // (1 << (i + 1))

    e_diag = np.array([1+0j for length in range(diag_length)])
    e = diags(e_diag)

    pow5 = pow5_list(diag_length)
    omiga_diag = np.array(dft_list(diag_length, pow5))
    omiga = diags(omiga_diag)

    combined_matrix0 = hstack([e, omiga])
    combined_matrix1 = hstack([e, -omiga])
    block = vstack([combined_matrix0, combined_matrix1])
    
    matrix = block_diag([block] * block_num)

    V_list[i] = matrix

    # print(matrix.toarray())

s2c_V_list = V_list # V_0

c2s_V_list = [0.5 * np.conj(V_list[i].T) for i in range(depth-1, -1, -1)] ## V_0_inv

V = V_list[0]
for i in range(1, depth):
    V =  V @ V_list[i]
# print(V.toarray())

V_inv_s2c = c2s_V_list[0]
for i in range(1, depth):
    # print(c2s_V_list[i].toarray())
    V_inv_s2c =  V_inv_s2c @ c2s_V_list[i]
# print(V_inv_s2c.toarray())

# print((V*V_inv_s2c).toarray())

In [8]:
## dense to diag

def res_and_ext(d0, d1):
    return np.hstack((d0, np.flip(d1)))

def ext_diag(d0, d1):
    return np.hstack((d0, d1))

def is_zero_diag(d0, d1):
    l = True
    r = True
    if d0 is not None:
        l = np.all(d0 == 0)
    if d1 is not None:
        r = np.all(d1 == 0)
    return l and r

def dense_to_diag(V_list, r=2, is_res = True):
    # r = 2
    if is_res:
        rows = depth
        cols_rest = 2 * r - 1
        cols_last = r
        diags = [[0 for _ in range(cols_rest)] for _ in range(rows-1)]
        diags.extend([[0 for _ in range(cols_last)]])

        diags_index = [[0 for _ in range(cols_rest)] for _ in range(rows-1)]
        diags_index.extend([[0 for _ in range(cols_last)]])
    else:
        rows = depth
        cols_0 = r
        cols_rest = 2 * r - 1
        diags = [[0 for _ in range(cols_0)]]
        diags.extend([[0 for _ in range(cols_rest)] for _ in range(rows-1)])

        diags_index = [[0 for _ in range(cols_0)]]
        diags_index.extend([[0 for _ in range(cols_rest)] for _ in range(rows-1)])

    for i in range(depth):
        print('depth: ', i)
        V_temp = V_list[i]

        rows = V_temp.shape[0]
        index = 0
        for n in range(rows):
            if n == 0:
                diag = V_temp.diagonal()
                # print(diag)
                if not is_zero_diag(diag, None):
                    # print(index)
                    diags[i][index] = diag
                    diags_index[i][index] = n
                    # print(diags_index[i][index]) 
                    # print(diags[i][index]) 
                    index = index + 1
            else:
                diag0 = V_temp.diagonal(n)
                diag1 = V_temp.diagonal(n-rows)
                # print(diag0)
                # print(diag1)
                if not is_zero_diag(diag0, diag1):
                    # print(index)
                    no_zero = ext_diag(diag0, diag1)
                    diags[i][index] = no_zero
                    diags_index[i][index] = n
                    # print(diags_index[i][index]) 
                    # print(diags[i][index])
                    index = index + 1

    return diags, diags_index

c2s_diags, c2s_diags_index = dense_to_diag(c2s_V_list, 2, True)
s2c_diags, s2c_diags_index = dense_to_diag(s2c_V_list, 2, False)

# print("print")
# for d in range(depth):
#     print("depth: ", d)
#     for i in range(0, len(c2s_diags[d])):
#         print(c2s_diags[d][i])

depth:  0
depth:  1
depth:  2
depth:  3
depth:  4
depth:  5
depth:  6
depth:  7
depth:  8
depth:  9
depth:  10
depth:  0
depth:  1
depth:  2
depth:  3
depth:  4
depth:  5
depth:  6
depth:  7
depth:  8
depth:  9
depth:  10


In [9]:
class ComplexNumberFileManager:
    """
    一个用于保存和加载复数数据的类。
    """

    def __init__(self, filename):
        self.filename = filename

    def save_to_file(self, complex_list):
        """
        将复数列表保存到文件中，实部和虚部分开保存。

        参数:
        complex_list (list): 要保存的复数列表。
        """
        try:
            with open(self.filename, "w") as file:
                for complex_num in complex_list:
                    real_part = complex_num.real
                    imag_part = complex_num.imag
                    file.write(f"{real_part} {imag_part}\n")
            print(f"复数列表已成功保存到文件 {self.filename}")
        except Exception as e:
            print(f"保存复数列表到文件时出错: {e}")

    def load_from_file(self):
        """
        从文件中加载复数列表。

        返回:
        list: 加载的复数列表。
        """
        complex_list = []
        try:
            with open(self.filename, "r") as file:
                for line in file:
                    real_part, imag_part = map(float, line.split())
                    complex_num = complex(real_part, imag_part)
                    complex_list.append(complex_num)
            print(f"复数列表已成功从文件 {self.filename} 加载")
        except Exception as e:
            print(f"从文件加载复数列表时出错: {e}")

        return complex_list

# 示例使用
# filename = "complex_numbers.txt"
# manager = ComplexNumberFileManager(filename)

# 保存复数列表
# complex_list = [1+1j, 2+2j, 3+3j, 4+4j]
# manager.save_to_file(complex_list)

# 从文件加载复数列表
# loaded_complex_list = manager.load_from_file()
# print(f"加载的复数列表：{loaded_complex_list}")


In [10]:
file_path = '/home/syt/data/phantom/file/diag_matrix/python/c2s/'
for i in range(depth):
    for j in range(len(c2s_diags[i])):
        filename = file_path + "host_C2SDFTMatrix_" + str(i) + "_" + str(j)+ "_" + str(c2s_diags_index[i][j]) + "_.txt"
        manager = ComplexNumberFileManager(filename)
        manager.save_to_file(c2s_diags[i][j])

复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_1_1_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_2_2047_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_1_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_1_1_2_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_1_2_2046_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_2_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_2_1_4_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_2_2_2044_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_3_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c

In [11]:
file_path = '/home/syt/data/phantom/file/diag_matrix/python/s2c/'
for i in range(depth):
    for j in range(len(s2c_diags[i])):
        filename = file_path + "host_S2CDFTMatrix_" + str(i) + "_" + str(j)+ "_" + str(s2c_diags_index[i][j]) + "_.txt"
        manager = ComplexNumberFileManager(filename)
        manager.save_to_file(s2c_diags[i][j])

复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_0_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_0_1_1024_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_1_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_1_1_512_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_1_2_1536_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_2_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_2_1_256_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_2_2_1792_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_3_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/s2c/host_S2CDFTMatrix_3_1_128_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/py

In [13]:
file_path = '/home/syt/data/phantom/file/diag_matrix/python/c2s/'
for i in range(depth):
    for j in range(len(c2s_diags[i])):
        filename = file_path + "host_C2SDFTMatrix_" + str(i) + "_" + str(j)+ "_" + str(c2s_diags_index[i][j]) + "_.txt"
        manager = ComplexNumberFileManager(filename)
        manager.save_to_file(c2s_diags[i][j])

复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_1_1_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_2_2047_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_1_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_1_1_2_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_1_2_2046_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_2_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_2_1_4_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_2_2_2044_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_3_0_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/diag_matrix/python/c

In [26]:
import numpy as np

def compare_complex_arrays_with_precision(arr1, arr2, tol=1e-5):
    """
    比较两个复数数组，相同则返回True，不同则输出不同的位置。
    考虑浮点精度。

    参数:
    arr1, arr2: 两个要比较的复数数组
    tol: 浮点精度容差

    返回:
    bool: 如果相同返回True，否则返回False并输出不同的位置
    """
    # if arr1.shape != arr2.shape:
    #     print("数组形状不同")
    #     return False

    # 比较两个数组，考虑浮点精度
    comparison = np.isclose(arr1, arr2, atol=tol)
    if np.all(comparison):
        return True
    else:
        # 找出不同的位置
        different_positions = np.where(comparison == False)
        print("不同的位置：", different_positions)
        return False

loaded_complex_list0 = []
loaded_complex_list1 = []

file_path0 = '/home/syt/data/phantom/file/diag_matrix/cpp/c2s/'
file_path1 = '/home/syt/data/phantom/file/diag_matrix/python/c2s/'

for i in range(depth):
    for j in range(len(c2s_diags[i])):
        filename0 = file_path0 + "host_C2SDFTMatrix_" + str(depth - 1 - i) + "_" + str(j)+ "_" + str(c2s_diags_index[i][j]) + "_.txt"
        filename1 = file_path1 + "host_C2SDFTMatrix_" + str(i) + "_" + str(j)+ "_" + str(c2s_diags_index[i][j]) + "_.txt"

        manager0 = ComplexNumberFileManager(filename0)
        manager1 = ComplexNumberFileManager(filename1)

        loaded_complex_list0 = manager0.load_from_file() 
        loaded_complex_list1 = manager1.load_from_file() 

        # print(loaded_complex_list0.shape)
        # print(loaded_complex_list1.shape)

        result = compare_complex_arrays_with_precision(loaded_complex_list0, loaded_complex_list1)
        print(result)

file_path0 = '/home/syt/data/phantom/file/diag_matrix/cpp/s2c/'
file_path1 = '/home/syt/data/phantom/file/diag_matrix/python/s2c/'

for i in range(depth):
    for j in range(len(s2c_diags[i])):
        filename0 = file_path0 + "host_S2CDFTMatrix_" + str(i) + "_" + str(j)+ "_" + str(s2c_diags_index[i][j]) + "_.txt"
        filename1 = file_path1 + "host_S2CDFTMatrix_" + str(i) + "_" + str(j)+ "_" + str(s2c_diags_index[i][j]) + "_.txt"

        manager0 = ComplexNumberFileManager(filename0)
        manager1 = ComplexNumberFileManager(filename1)

        loaded_complex_list0 = manager0.load_from_file() 
        loaded_complex_list1 = manager1.load_from_file() 

        # print(loaded_complex_list0.shape)
        # print(loaded_complex_list1.shape)

        result = compare_complex_arrays_with_precision(loaded_complex_list0, loaded_complex_list1)
        print(result)

print('done.')

复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/cpp/c2s/host_C2SDFTMatrix_10_0_0_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_0_0_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/cpp/c2s/host_C2SDFTMatrix_10_1_1_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_1_1_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/cpp/c2s/host_C2SDFTMatrix_10_2_2047_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_0_2_2047_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/cpp/c2s/host_C2SDFTMatrix_9_0_0_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_1_0_0_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/cpp/c2s/host_C2SDFTMatrix_9_1_2_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/diag_matrix/python/c2s/host_C2SDFTMatrix_1_1_2_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/di